In [1]:
## Attaches burn probabilities to the ZTRAX data
## Uses lat and long provided 
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import glob
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show

In [2]:
## Get US Map for context and select Western United States
USMAP = gpd.read_file('/data/yoder/DensityProject/cb_2018_us_state_500k.shp')
West = ['WA', 'OR', 'CA', 'NV','AZ','NM', 'UT','CO','WY','ID','MT']

## Get map(s) of individual states if desired
WestStates = USMAP[USMAP['STUSPS'].isin(West)]

##Convert Coordinate Reference System to US Natinoal Atlas Equal Area
WestStates = WestStates.to_crs("EPSG:2163")

In [3]:
scratchpath = '/scratch/user/joshua.olsen/20210428_153601/'
os.chdir(scratchpath)
## List all tables included in Zillow Variable names excel file
ZTransLayout = pd.read_excel('Layout.xlsx','ZTrans')
ZTransLayout['TableName'].unique()
ZAsmtLayout = pd.read_excel('Layout.xlsx','ZAsmt')

## Create dataframes from each table name in the Zillow table dictionary
    ## Strip 'ut' from column names
T = {}
for name,group in ZTransLayout.groupby('TableName'):
    T[str(name)[2:]] = group
A = {}
for name, group in ZAsmtLayout.groupby('TableName'):
    A[str(name)[2:]] = group
    
    
## Dictionary of Western 11 states using Zillows numbering: 
states = {'AZ':'04',
          'CA':'06',
          'CO':'08',
          'ID':'16',
          'MT':'30',
          'NV':'32',
          'NM':'35',
          'OR':'41',
          'UT':'49',
          'WA':'53',
          'WY':'56'}

In [4]:
#Change this value to the state and table you care about
# State to use
state = states['CO']
state_map = USMAP[USMAP['STUSPS']==state]
#Transaction tables to use
Ttables = ['Main', 'PropertyInfo']
# Assessment tables to use
Atables = ['Main','Garage', 'Building']

In [5]:
## Fields to use from transaction tables
Tfields = {'Main': ['TransId','State','County','DocumentDate','SignatureDate',
                    'RecordingDate','FIPS','DataClassStndCode',
                    'DocumentTypeStndCode','IntraFamilyTransferFlag',
                    'LoanTypeStndCode','PropertyUseStndCode',
                    'SalesPriceAmount', 'LoanAmount'],
            'PropertyInfo':['TransId','ImportParcelID','AssessorParcelNumber',
                            'PropertyFullStreetAddress','PropertyCity',
                            'PropertyState','PropertyAddressLatitude',
                            'PropertyAddressLongitude']}
# Fields to use from Assessment tables
Afields = {'Main': ['RowID','ImportParcelID','LotSizeSquareFeet'],
          'Garage':['RowID','GarageStndCode','GarageAreaSqFt'],
          'Building':['RowID','ArchitecturalStyleStndCode','BuildingClassStndCode','BuildingQualityStndCode',
                       'BuildingConditionStndCode','EffectiveYearBuilt','YearBuilt','YearRemodeled',
                       'NoOfStories','TotalRooms','TotalBedrooms','TotalKitchens',
                       'FullBath','ThreeQuarterBath','HalfBath','QuarterBath','TotalActualBathCount',
                       'TotalBathPlumbingFixtures','RoofCoverStndCode', 'RoofStructureTypeStndCode',
                       'HeatingTypeorSystemStndCode','AirConditioningTypeorSystemStndCode',
                       'FoundationTypeStndCode','ElevatorStndCode','FireplaceFlag',
                       'FireplaceNumber','WaterStndCode','SewerStndCode','TimeshareStndCode',                       ]} 
## Throws error if these are included 'FireplaceTypeStndCode','StoryTypeCode'

In [6]:
%%time
## Load Ztrans data into dictionary of table entries that will be merged
dfs = {}
keys = Ttables
for i in keys:
    # Populate dictionary with desired tables for the given state
    dfs[i] = pd.read_csv(f'{state}/ZTrans\{i}.txt',sep = '|', header=None,names=T[i]['FieldName'].tolist(), encoding='latin1',usecols=Tfields[i]) 

<decorator-gen-54>:2: DtypeWarning: Columns (17,18,30) have mixed types.Specify dtype option on import or set low_memory=False.
<decorator-gen-54>:2: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min 7s, sys: 5.62 s, total: 1min 12s
Wall time: 1min 12s


In [7]:
%%time
## Load Assessor data

dfA = {}
keys = Atables
for j in keys:
    dfA[j] = pd.read_csv(f'{state}/ZAsmt\{j}.txt',sep='|', header=None,names=A[j]['FieldName'].tolist(),encoding='latin1',usecols=Afields[j])

<decorator-gen-54>:2: DtypeWarning: Columns (9,10,12,29,30,31,32,33,34,35,38,39) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 18.2 s, sys: 1.62 s, total: 19.8 s
Wall time: 19.8 s


In [8]:
## Merge Trans tables and Asmt tables
from functools import reduce
Trans = reduce(lambda left,right: pd.merge(left,right,on='TransId',how='left'),dfs.values())
Asmt = reduce(lambda left, right:pd.merge(left,right,on='RowID',how='left'),dfA.values())
df = pd.merge(Trans,Asmt,on='ImportParcelID',how='left')

In [9]:
## Select only valid transactions
UnwantedLoanCodes =  ['AC','CT','CM','CS','CC','CL','DP','FO','FE','HE','LC','EB','EX','MD','NA','NP','FA','RE','RM','SM','SE','TR']
WantedLoanCodes =  ['AS','BL','CE','FM','PM','RD','SL']
DocumentTypeStndCodeDrop= ['CRDE','JTDE','QCDE','RRDE','VLDE']
DocumentTypeStndCodeKeep= ['CPDE','IDDE','PRDE','WRDE']

st = df.loc[(df['SalesPriceAmount'] > 0) &
            (~df['LoanTypeStndCode'].isin(UnwantedLoanCodes)) &
            (pd.isnull(df['IntraFamilyTransferFlag'])) &
            (df['PropertyUseStndCode']=='SR') &
            (df['DocumentTypeStndCode'].isin(DocumentTypeStndCodeKeep)) & 
            (df['TotalBedrooms'] >=1)].copy()

In [10]:
st

,TransId,FIPS,State,County,DataClassStndCode,RecordingDate,DocumentTypeStndCode,DocumentDate,SignatureDate,SalesPriceAmount,...,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,TimeshareStndCode
38,97922632,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,NaN,91000.0,...,GBL,BB,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
39,97922632,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,NaN,91000.0,...,GBL,BB,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
76,97922653,8001,CO,ADAMS,D,1993-09-01,WRDE,NaN,NaN,55000.0,...,HIP,HW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,97922694,8001,CO,ADAMS,H,1993-09-02,WRDE,NaN,NaN,160000.0,...,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
142,97922694,8001,CO,ADAMS,H,1993-09-02,WRDE,NaN,NaN,160000.0,...,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19847061,442203817,8123,CO,WELD,H,2018-09-17,WRDE,2018-09-14,NaN,256000.0,...,NaN,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19847067,442203821,8123,CO,WELD,H,2018-09-17,WRDE,2018-09-14,NaN,254000.0,...,NaN,CE,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19847068,442203821,8123,CO,WELD,H,2018-09-17,WRDE,2018-09-14,NaN,254000.0,...,NaN,CE,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19847072,442203824,8123,CO,WELD,H,2018-09-17,WRDE,2018-09-14,NaN,339500.0,...,NaN,CE,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Make geodataframe

st = gpd.GeoDataFrame(st, geometry = gpd.points_from_xy(st.PropertyAddressLongitude, st.PropertyAddressLatitude))
st_proj = st.copy()
st_proj.crs="epsg:4326"
st_proj=st_proj.to_crs("EPSG:2163")
st_proj.crs

In [ ]:
import rasterio
from rasterstats import zonal_stats, point_query
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
## Reproject raster and load it

dst_crs ="EPSG:2163"

with rasterio.open('/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    with rasterio.open('/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
                   reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
                
src = rasterio.open('/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif')

In [ ]:
fig, ax = plt.subplots()
show(src,ax = ax)
st_proj.plot(ax=ax, color ='red')

In [ ]:
pts = st_proj.copy()

In [ ]:
st_proj.shape

In [ ]:
## Select only valid geometry (note .isna() does not appear to catch point (nan,nan))
st_pts = st_proj[st_proj['geometry'].is_valid]

In [ ]:
st_pts.shape

In [ ]:
# pts = CO_pts.head(100000).copy()
pts = st_pts.copy()

In [ ]:
%%time
## Add burn probability that the location is within (landfire uses 270 meter pixels)
pts['BP'] = point_query(pts, '/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif')

In [ ]:
%%time
## Add burn probability out to 1600 meters
pts['point_geometry'] = pts['geometry'].copy()
pts['geometry'] = pts.buffer(1600)
buff = pd.DataFrame(zonal_stats(pts, 
                             '/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif',
                            stats=['max','mean']))
buff.columns = [f'BP1600_'+str(col) for col in buff.columns]

buff.reset_index(drop=True,inplace=True)
pts.reset_index(drop=True,inplace=True)
pts = pd.concat([pts,buff],axis=1)
pts['geometry'] = pts['point_geometry'].copy()

In [ ]:
BurnCols = ['BP','BP1600_max','BP1600_mean']

In [ ]:
basemap = state_map.plot(edgecolor='black',color='white')
pts.plot(ax= basemap)

In [ ]:
## Create a copy of everything to merge with fires
## For Colorado base memory usage is 471mb with 1,374,190 rows
ST_points = pts.copy()

In [ ]:
## Add fire perimeter data
import geopandas as gpd
Fireperims = gpd.read_file('/data/yoder/DensityProject/mtbs_perimeter_data/mtbs_perims_DD.shp')
Fireperims = Fireperims.to_crs("EPSG:2163")

In [ ]:
state_map = state_map[['STUSPS', 'geometry']]
Buffed_state = state_map.copy()
Buffed_state['geometry'] = Buffed_state.buffer(100000)
ST_fires = gpd.sjoin(Fireperims, Buffed_state, op='within')
ST_fires = ST_fires.drop(['index_right'], axis=1)
ST_fires['Buffed_geometry100km'] = ST_fires.buffer(100000)
ST_fires['MTBS_geometry'] = ST_fires['geometry'].copy()
ST_fires['geometry'] = ST_fires['Buffed_geometry100km'].copy()

In [ ]:
# ST_samp = pts.sample(n=10000)

In [ ]:
basemap = state_map.plot(edgecolor='black',color='white')
ST_fires.plot(ax=basemap, color='red')
ST_points.plot(ax= basemap, color='blue')

In [ ]:
%%time
ST_Samp_with_Fires = gpd.sjoin(ST_points, ST_fires, how='left')

In [ ]:
def get_distance(row):
    distance = row.geometry.distance(row.MTBS_geometry)
    return distance

In [ ]:
%%time
ST_Samp_with_Fires['Distance'] = ST_Samp_with_Fires.apply(lambda row: get_distance(row),axis=1)

In [ ]:
# ST_Samp_with_Fires['Distance'].describe()

In [ ]:
# pricy = ST_Samp_with_Fires.loc[ST_Samp_with_Fires['SalesPriceAmount']>=10000000]

In [ ]:
# pricy.drop_duplicates(subset=['TransId'])

In [ ]:
ST_Samp_with_Fires.describe()